In [1]:
import os
import numpy as np
import random
import glob
from datetime import datetime
import logging
import sys
import h5py
import csv
import time
import gc
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Activation, Input, Concatenate, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import callbacks, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

2024-11-07 15:20:18.905485: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-07 15:20:18.996919: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-07 15:20:19.021903: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-07 15:20:19.178389: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-07 15:20:20.453119: W tensorflow/compiler/tf2

In [2]:
def parse_file_pointer(fp, tam):
    lines = [ll.strip() for ll in fp]
    ii = 0
    labels = []
    res = []
    cli = []
    numLinhas = 0
    while ii < len(lines):
        line = lines[ii]
        #contando o numero de vertices do grafo
        if "cliqueatual" not in line:
            ii += 1
            numLinhas += 1
            continue

        #pegando a clique atual
        if ii+1 >= len(lines):
            break
        line = line[3:]
        spritado = line.split()
        clique = [int(elem) for elem in spritado[1:]]
        if(numLinhas < tam):
            dif = tam - numLinhas
            clique.extend([0]*dif)
        cli.append(clique)

        #criando o vetor de movimento
        line = lines[ii+1]
        sp = line.split()
        mv = int(sp[-1])
        label = [0] * tam
        label[mv] = 1
        labels.append(label)

        #lendo o grafo
        cells = []
        for tt in range(numLinhas, 0, -1):
            cell_line = lines[ii - tt][3:]
            cells.extend([int(float(cc)) for cc in cell_line.split(", ")])
            if(numLinhas < tam):
                dif = tam - numLinhas
                cells.extend([0]*dif)
        while len(cells) < tam * tam:
            cells.extend([0]*tam)

        #cells = np.reshape(cells,((tam,  -1)))
        #cells = np.transpose(cells)
        #cells = np.reshape(cells, -1)
        res.append(cells)
        ii += (numLinhas+2)
    labels_v = list(range(len(labels),0, -1))
    return (res, cli, labels, labels_v)

In [3]:
def parse_dir(ddir, tam):
    res = []
    cli = []
    labels = []
    labels_v = []
    random.seed(42)
    files = sorted([os.path.basename(ii) for ii in glob.glob("{0}/*.dimacs".format(ddir))])
    random.shuffle(files)
    random.seed()
    i = 0
    for ff in files:
        with open(os.path.join(ddir,ff), 'r') as fp:
            rr, cc, ll, ll_v = parse_file_pointer(fp, tam)
            res.extend(rr)
            cli.extend(cc)
            labels.extend(ll)
            labels_v.extend(ll_v)
        i+=1
        if i > 5:
            break
    return res, cli, labels, labels_v

In [4]:
class printbatch(callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        logging.info("Epoch: "+ str(epoch))
    def on_epoch_end(self, epoch, logs={}):
        logging.info(logs)

In [5]:
class LoggerWriter:
    def __init__(self, level):
        self.level = level

    def write(self, message):
        if message != '\n':
            self.level(message)

    def flush(self):
        self.level(sys.stderr)

In [6]:
def learn(data, clique, labels, tam, output_path, shared_layer_multipliers, layer_multipliers, batch_size, learning_rate):
    
    shared_layer_multipliers = [x for x in shared_layer_multipliers if x != 0]

    # Definir camadas de entrada
    graph_input = [Input(shape=(tam,)) for _ in range(tam)]  # Matriz de adjacência
    clique_input = Input(shape=(tam,))  # Vetor de clique

    # Camadas densas compartilhadas
    graph_input_processed = graph_input
    clique_input_processed = clique_input
    for i in range(len(shared_layer_multipliers)):
        shared_dense_graph = Dense(tam * shared_layer_multipliers[i], activation='relu')
        shared_dense_clique = Dense(tam * shared_layer_multipliers[i], activation='relu')
        graph_input_processed = [shared_dense_graph(inp) for inp in graph_input_processed]
        clique_input_processed = shared_dense_clique(clique_input_processed)
    
    # Concatenar os vetores processados
    merged_vector = Concatenate(axis=-1)(graph_input_processed + [clique_input_processed])
    
    #camadas internas
    layer = merged_vector
    for i in range(len(layer_multipliers)):
        layer = Dense(tam*layer_multipliers[i],activation='relu')(layer)

    #camada de saida
    output_layer = Dense(tam, activation='softmax')(layer)
    
    #compilar modelo
    model = Model(inputs=[graph_input, clique_input], outputs=output_layer)
    adam = optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

    #treinar modelo
    now = datetime.now()
    model.fit([np.hsplit(data, tam), clique], labels, epochs=1000, batch_size=batch_size,validation_split=0.2,verbose=2,
              callbacks=[printbatch(), EarlyStopping(monitor='val_loss', patience=50, verbose=0), ModelCheckpoint(os.path.join(output_path, "models",
                            "dnn_model_" + str(tam) + "_"+ str(now.day) + "." + str(now.month) + "." + str(now.year) + "_"
                            + "_{epoch:02d}-{val_loss:.2f}" + ".h5"),
                            monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')])
    model.save(os.path.join(output_path, "dnn_model_" + str(tam)+"_"+ str(now.day) + "." + str(now.month) + "." + str(now.year) +"_"+ ".h5"))

In [7]:
def learn_value(data, clique, labels, tam, output_path, shared_layer_multipliers, layer_multipliers, batch_size, learning_rate):
   
    shared_layer_multipliers = [x for x in shared_layer_multipliers if x != 0]

    # Definir camadas de entrada
    graph_input = [Input(shape=(tam,)) for _ in range(tam)]  # Matriz de adjacência
    clique_input = Input(shape=(tam,))  # Vetor de clique

    # Camadas densas compartilhadas
    graph_input_processed = graph_input
    clique_input_processed = clique_input
    for i in range(len(shared_layer_multipliers)):
        shared_dense_graph = Dense(tam * shared_layer_multipliers[i], activation='relu')
        shared_dense_clique = Dense(tam * shared_layer_multipliers[i], activation='relu')
        graph_input_processed = [shared_dense_graph(inp) for inp in graph_input_processed]
        clique_input_processed = shared_dense_clique(clique_input_processed)
    
    # Concatenar os vetores processados
    merged_vector = Concatenate(axis=-1)(graph_input_processed + [clique_input_processed])
    
    #camadas internas
    layer = merged_vector
    for i in range(len(layer_multipliers)):
        layer = Dense(tam*layer_multipliers[i],activation='relu')(layer)

    #camada de saida
    output_layer = Dense(1)(layer)

    #compilar modelo
    model = Model(inputs=[graph_input, clique_input], outputs=output_layer)
    adam = optimizers.Adam(lr=learning_rate)
    model.compile(optimizer=adam, loss='mse', metrics=['mae'])

    #treinar modelo
    now = datetime.datetime.now()
    model.fit([np.hsplit(data, tam), clique], labels, epochs=1000, batch_size=batch_size, validation_split=0.2, verbose=2,
              callbacks=[printbatch(), EarlyStopping(monitor='val_loss', patience=50, verbose=0), ModelCheckpoint(os.path.join(output_path, "models",
                            "dnn_value_model_" + str(tam) + "x" +"_"+ str(now.day) + "." + str(now.month) + "." + str(now.year) + "_" +
                            "_{epoch:02d}-{val_loss:.2f}" + ".h5"), monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False,
                            mode='auto')])
    model.save(os.path.join(output_path, "dnn_value_model_"+str(tam)+"_"+str(now.day)+"."+str(now.month)+"."+str(now.year)+"_"+".h5"))

In [8]:
def main():
    output_path = "modelos" #caminha onde é salvo o modelo
    labeled_data_dir = "train_graphs" #caminho dos dados para treinar o modelo
    param_v_a_1 = [4, 3, 2] #camadas compartilhadas rede bound
    param_v_a_2 = [3, 2, 2] #camadas ocultas
    param_p_a_1 = [6, 4, 3] #camadas compartilhadas rede brach
    param_p_a_2 = [9, 6, 2] #camadas ocultas
    param_p_b = 512 #batch size
    param_v_b =  512
    param_p_l = 0.001 #taxa de aprendizado
    param_v_l = 0.001 
    tam = 250
    use_value_model = True #se vai treinar a rede de bound

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    #treinar rede branch
    res, clique, labels, labels_v = parse_dir(labeled_data_dir, tam)
    print("leu tudo")
    learn(np.array(res), np.array(clique), np.array(labels), tam, output_path, param_p_a_1, param_p_a_2, param_p_b, param_p_l)
    print("rede 1 treinada")
    #treinar rede bound
    if use_value_model:
        learn_value(np.array(res), np.array(clique), np.array(labels_v), tam, output_path, param_v_a_1, param_v_a_2, param_v_b, param_v_l)
        print("rede 2 treinada")

main()

leu tudo


I0000 00:00:1731003753.182851    6063 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1731003753.354421    6063 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1731003753.354471    6063 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1731003753.358751    6063 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1731003753.358883    6063 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2] name: 